# 蔵本モデルのシミュレーション

蔵本モデルの平均場モデル
$$ \dot{\theta_k} = \omega_k + Kr\sin(\Psi - \theta_k), \quad r e^{i\Psi} = \frac{1}{N} \sum_{j = 1}^{N} e^{i \theta_k} $$
のシミュレーションを行う．

In [3]:
import cmath
import numpy as np
from scipy.integrate import solve_ivp
from matplotlib import pyplot as plt
from matplotlib import rc # imported for showing the animation on colaboratory
from IPython.display import HTML # imported for showing the animation on colaboratory
import matplotlib as mpl
mpl.rcParams["mathtext.fontset"] = "stix"
mpl.rcParams["animation.embed_limit"] = 2**128
from matplotlib.animation import FuncAnimation
from matplotlib.patches import Arrow

ModuleNotFoundError: No module named 'numpy'

## 蔵本モデルの定義

In [ ]:
def kuramoto_model(t, v, N, K, omega):
    '''
    Kuramoto model (mean field model)
    '''
    op = np.exp(1j * v).sum() / N # Kuramoto Order Parameter
    r, psi = cmath.polar(op) # absolute value and phase
#    r = np.abs(op) # absolute value
#    psi = np.angle(op) # phase
    vdot = omega + K * r * np.sin(psi - v)
    return vdot

: 

## パラメータなどの設定

In [ ]:
N = 1600 # number of oscillators
K = 3 # coupling constant
seed = 2021 # random seed

t_end = 1000 # calculation terminates at t = t_end
n_point = 1000 # number of data to animate

: 

## 初期状態をランダムに生成

In [ ]:
rng = np.random.default_rng(seed=seed) # generator
# initial values of omega follow normal distribution with mean = 0 and sd = 1
omega = rng.normal(loc=0, scale=1, size=N)
# initial values of theta follow uniform distribution
ic = rng.random(size=(N,)) * 2 * np.pi

: 

## 数値的に解き，データを記録する
振動子数を増やすと解くのに時間がかかる．毎回計算すると大変なので，csvやnpzファイルとして計算結果を保存して，それを読み込んで図や動画を作成した方がおそらく良い．

ここでは簡潔にするために，計算した結果をそのまま描画に使う．

In [ ]:
t_eval = np.linspace(0, t_end, n_point)
# solve the initial value problem
res = solve_ivp(fun=kuramoto_model, args=(N, K, omega,), y0=ic, 
                t_span=(0, t_end), t_eval=t_eval, atol=1e-6, rtol=1e-3)

t = res.t # history of time
theta_hist = res.y # history of theta
op_hist = np.exp(1j * res.y).sum(axis=0) / N # history of order parameter
r_hist = np.abs(op_hist) # history of r(t)
psi_hist = np.angle(op_hist) # history of Psi(t)

: 

## 結果を図示する
左側に$r$と$\Psi$の時系列，右側に$\theta_k$の時間発展を動画で示す．

In [ ]:
def update(tpl, m1, m2, ax, sc, txt,):
    '''
    updates some elements in the figure.
    This method will be called in FuncAnimation().
    '''
    t, theta, r, psi = tpl

    m1.set_data(t, r)
    m2.set_data(t, psi / np.pi)

    data = np.hstack((np.cos(theta)[:, np.newaxis], np.sin(theta)[:, np.newaxis]))
    sc.set_offsets(data)
    txt.set_text(rf'$t = {t:.2f}$')

    ax.patches = []
    arrow = Arrow(x=0, y=0, dx=r * np.cos(psi), dy=r * np.sin(psi), width=0.1, color='k')
    a = ax.add_patch(arrow)
    return m1, m2, sc, txt, a,

: 

描画のメインの処理．  
Jupyter NotebookやColaboratoryを使っているか，.pyファイルとして実行するかで，最後の部分の処理が変わる(適宜コメントアウトをつけたり外したりしてください)．

In [ ]:
fig, axes = plt.subplots(ncols=2, figsize=(8, 4), dpi=120)
fig.tight_layout(rect=[0,0,1,0.9])
fig.suptitle(rf'Kuramoto Model with $N = {N}$, $K = {K}$') # title for entire figure

# initialize the figure

####################
# plot time-series of r and Psi on the left
####################
ax = axes[0]
ax.set_title(r'Time Evolution of $r$, $\Psi$')
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$r$')
ax.set_ylim((0, 1))

# second y-axis on the right side
twin = ax.twinx()
twin.set_ylabel(r'$\Psi / \pi$')
twin.set_ylim((-1.5, 1.5))

# plot data
pl1, = ax.plot(t, r_hist, lw=0.8, c='r', zorder=5, label=r'$r$')
pl2, = twin.plot(t, psi_hist / np.pi, lw=0.8, c='c', zorder=3, label=r'$\Psi / \pi$')
ax.legend(handles=[pl1, pl2])

# show markers
m1, = ax.plot(t[0], r_hist[0], ls='none', marker='o', c='r')
m2, = twin.plot(t[0], psi_hist[0] / np.pi, ls='none', marker='o', c='c')

####################
# animate the time evolution of theta_k
####################
ax = axes[1]

# axes settings
p = np.linspace(0, 2 * np.pi, 50)
ax.tick_params(axis='both', left=False, bottom=False, labelleft=False, labelbottom=False)
ax.set_aspect('equal')

ax.set_title(r'Time Evolution of phase $\theta_k$')
ax.plot(np.cos(p), np.sin(p), lw=0.5, c='k', zorder=1)

# show the time and data
txt = ax.text(x=0.5, y=-1, s=r'$t = 0$')
sc = ax.scatter(np.cos(theta_hist[:, 0]), np.sin(theta_hist[:, 0]), s=5, c=omega, cmap='jet', edgecolors='k', linewidth=0.1, zorder=3)
plt.colorbar(sc, ax=ax, fraction=0.046, pad=0.04, label=r'$\omega_k$')

# draw an arrow indicating r and Psi
arrow = Arrow(x=0, y=0, dx=r_hist[0] * np.cos(psi_hist[0]), dy=r_hist[0] * np.sin(psi_hist[0]), width=0.1, color='k')
a = ax.add_patch(arrow)

# create an animation
## if you are running this script as .ipynb file or on colaboratory (only show first 240 frames)
#ani = FuncAnimation(fig, update, frames=zip(t, theta_hist.T, r_hist, psi_hist), fargs=(m1, m2, ax, sc, txt,), interval=1000 / 30, blit=True, save_count=240)

## use instead these lines if you are running this script as .py file
#ani = FuncAnimation(fig, update, frames=zip(t, theta_hist.T, r_hist, psi_hist), fargs=(m1, m2, ax, sc, txt,), interval=1000 / 30, blit=True)
#plt.show()


: 

FuncAnimation()で動画を描画するので，基本的には動画を再生しつつ描画処理が行われる(表示する前に動画作成を終える必要がない)．
.pyファイルとして実行する場合にはこれがうまくいくが，Jupyter NotebookやColaboratoryではうまくいかないので，`save_count`で描画するフレーム数を指定し，その分だけ動画が作成された後で再生できる．
ある程度長さのある動画を表示したい場合は.pyファイルとして実行する方が良さそう．

動画を保存したい場合には，`save_count`を指定して先に動画を作成する必要がある．
これには多少時間がかかる(Colaboratory上だと重すぎるのか，1000フレーム全体を使った動画は作成できないみたい)．

In [ ]:
# show the figure on jupyter notebook or colaboratory(it may take some time)
# comment these lines if you are running this script as a .py file
rc('animation', html='jshtml')
ani

: 

2021-09-22　石井秀昌